<a href="https://colab.research.google.com/github/HwangHanJae/BigDataPratice/blob/main/5_week.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#작업형1

데이터 출처 :https://www.kaggle.com/mirichoi0218/insurance/code(참고, 데이터 수정)

데이터 설명 : 의료비용 예측문제

Data url :https://raw.githubusercontent.com/Datamanim/datarepo/main/MedicalCost/train.csv

##Q1. 흡연자와 비흡연자 각각 charges의 상위 10% 그룹의 평균의 차이는?

In [47]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/MedicalCost/train.csv")

df = df.sort_values("charges", ascending=False)

no_10 = df.groupby("smoker")["charges"].count()[0] * 0.1
yes_10 = df.groupby("smoker")["charges"].count()[1] * 0.1

df.loc[df["smoker"] == "yes","charges"].head(int(yes_10)).mean() - df.loc[df["smoker"] == "no", "charges"].head(int(no_10)).mean()

29297.954548156144

In [53]:
#Answer
high = df.loc[df["smoker"] == "yes"]["charges"].quantile(0.9)
high2 = df.loc[df["smoker"] == "no"]["charges"].quantile(0.9)

mean_yes = df.loc[(df["smoker"] == "yes") & (df["charges"] >= high), "charges"].mean()
mean_no = df.loc[(df["smoker"] == "no") & (df["charges"] >= high2), "charges" ].mean()

print(mean_yes - mean_no)

29297.954548156144


#작업형2

데이터 출처 :https://www.kaggle.com/mirichoi0218/insurance/code(참고, 데이터 수정)

데이터 설명 : 의료비용 예측문제

​평가지표 : r2 score

trainData url : https://raw.githubusercontent.com/Datamanim/datarepo/main/MedicalCost/train.csv

testData url : https://raw.githubusercontent.com/Datamanim/datarepo/main/MedicalCost/test.csv

In [54]:
trainData = "https://raw.githubusercontent.com/Datamanim/datarepo/main/MedicalCost/train.csv"
testData = "https://raw.githubusercontent.com/Datamanim/datarepo/main/MedicalCost/test.csv"

import pandas as pd
train = pd.read_csv(trainData)
test = pd.read_csv(testData)

train

,age,sex,bmi,children,smoker,region,charges
0,64,female,39.330,0,no,northeast,14901.51670
1,47,female,27.830,0,yes,southeast,23065.42070
2,52,female,33.300,2,no,southwest,10806.83900
3,33,female,22.135,1,no,northeast,5354.07465
4,30,male,31.400,1,no,southwest,3659.34600
...,...,...,...,...,...,...,...
1065,39,male,24.510,2,no,northwest,6710.19190
1066,37,male,24.320,2,no,northwest,6198.75180
1067,18,female,26.315,0,no,northeast,2198.18985
1068,50,male,37.070,1,no,southeast,9048.02730


In [55]:
test

,age,sex,bmi,children,smoker,region
0,37,male,37.070,1,yes,southeast
1,63,male,31.445,0,no,northeast
2,21,male,31.255,0,no,northwest
3,54,female,28.880,2,no,northeast
4,60,female,18.335,0,no,northeast
...,...,...,...,...,...,...
263,50,male,30.970,3,no,northwest
264,18,female,31.920,0,no,northeast
265,18,female,36.850,0,no,southeast
266,21,female,25.800,0,no,southwest


In [56]:
train.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [57]:
test.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
dtype: int64

In [59]:
#데이터 one-hot-encoding
columns = ["sex","smoker","region"]

train = pd.get_dummies(train, columns)
test = pd.get_dummies(test, columns)

In [61]:
#타겟 데이터 분리
X = train.drop("charges", axis=1)
y = train["charges"]

In [64]:
#훈련데이터, 검증데이터 분리
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

#모델링
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_val)

from sklearn.metrics import r2_score

print(r2_score(y_val, y_pred))

0.8259880044219918


In [67]:
#평가
#submission 생성
import numpy as np
sub = {"charges" : np.zeros(test.shape[0])}
submission = pd.DataFrame(sub)

pred = rf.predict(test)
submission["charges"] = pred
submission

,charges
0,40624.766682
1,14234.532559
2,3115.753550
3,12387.693151
4,17900.364703
...,...
263,11303.201919
264,2179.322196
265,3023.128628
266,2085.162041


In [68]:
#제출
submission.to_csv("수험번호.csv", index = False)